## Autoencoder(Combined with CNN?)

In [1]:
import manga109api
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import skimage
%matplotlib inline

In [2]:
manga109_root = "../datasets/Manga109/Manga109_released_2021_12_30"
dataset = manga109api.Parser(manga109_root)

In [3]:
def data(is_train = True):
    """define a generator as input to training to not overload the RAM"""
    for book in dataset.books:
        for page in dataset.get_annotation(book)["page"]:
            count = random.randint(0, 19)
            if count % 20 == 0 and is_train: continue
            if count % 20 != 0 and (not is_train): continue
            width = page["@width"]
            height = page["@height"]
            #skip images that are larger than 2048 in at least one dimension
            if width > 2048 or height > 2048:
                continue
            image = tf.convert_to_tensor((skimage.io.imread("../datasets/Manga109/Manga109_released_2021_12_30/padded_text_stripped_images/" + book + "/%03d.jpg" % (page["@index"]))[np.newaxis])/255)
            yield image, image

In [4]:
"""covert generator to tensorflow dataset"""
train_data = tf.data.Dataset.from_generator(data,
                               output_signature=(
                                   tf.TensorSpec(shape=(None,2048,2048,3)),
                                   tf.TensorSpec(shape=(None,2048,2048,3))
                               ))
test_data = tf.data.Dataset.from_generator(lambda: data(False),
                               output_signature=(
                                   tf.TensorSpec(shape=(None,2048,2048,3)),
                                   tf.TensorSpec(shape=(None,2048,2048,3))
                               ))

In [7]:
"""create autoencoder neural network"""
class Autoencoder(tf.keras.Model):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = tf.keras.Sequential()
        for i in range(7):
            self.encoder.add(layers.Conv2D(32, (5, 5), activation="relu", padding="same"))
            self.encoder.add(layers.MaxPooling2D((2, 2), padding="same"))
        self.decoder = tf.keras.Sequential()
        for i in range(7):
            self.decoder.add(layers.Conv2DTranspose(32, (5, 5), strides=2, activation="relu", padding="same"))
        self.decoder.add(layers.Conv2D(3, (5, 5), activation="softmax", padding="same"))
    def call(self, _input):
        encoded = self.encoder(_input)
        decoded = self.decoder(encoded)
        return decoded

autoencoder = Autoencoder()
autoencoder.compile(optimizer="adam", loss="mean_squared_error")

In [8]:
autoencoder.fit(
    x=train_data,
    epochs=2,
    validation_data=test_data
)

Epoch 1/2
   1113/Unknown - 808s 720ms/step - loss: 0.2094

KeyboardInterrupt: 

In [10]:
book = "ARMS"
page = {}
page["@index"] = 4
plt.imshow(autoencoder.predict(skimage.io.imread("../datasets/Manga109/Manga109_released_2021_12_30/padded_text_stripped_images/" + book + "/%03d.jpg" % (page["@index"]))[np.newaxis])[0])

array([[[0.3341383 , 0.33354583, 0.3323159 ],
        [0.3341383 , 0.33354583, 0.3323159 ],
        [0.3341383 , 0.33354583, 0.3323159 ],
        ...,
        [0.3341383 , 0.33354583, 0.3323159 ],
        [0.3341383 , 0.33354583, 0.3323159 ],
        [0.3341383 , 0.33354583, 0.3323159 ]],

       [[0.3341383 , 0.33354583, 0.3323159 ],
        [0.3341383 , 0.33354583, 0.3323159 ],
        [0.3341383 , 0.33354583, 0.3323159 ],
        ...,
        [0.3341383 , 0.33354583, 0.3323159 ],
        [0.3341383 , 0.33354583, 0.3323159 ],
        [0.3341383 , 0.33354583, 0.3323159 ]],

       [[0.3341383 , 0.33354583, 0.3323159 ],
        [0.3341383 , 0.33354583, 0.3323159 ],
        [0.3341383 , 0.33354583, 0.3323159 ],
        ...,
        [0.3341383 , 0.33354583, 0.3323159 ],
        [0.3341383 , 0.33354583, 0.3323159 ],
        [0.3341383 , 0.33354583, 0.3323159 ]],

       ...,

       [[0.3341383 , 0.33354583, 0.3323159 ],
        [0.3341383 , 0.33354583, 0.3323159 ],
        [0.3341383 , 0

In [ ]:
#skimage.io.imread("../datasets/Manga109/Manga109_released_2021_12_30/padded_text_stripped_images/" + book + "/%03d.jpg" % (page["@index"])).shape

## References

[https://keras.io/examples/vision/autoencoder/](https://keras.io/examples/vision/autoencoder/)